# Selector Group Chat

SelectorGroupChat is a group chat similar to RoundRobinGroupChat, but with a model-based next speaker selection mechanism.

In [14]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('GEMINI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gemini-1.5-flash-8b', api_key=api_key)

In [15]:
planning_agent = AssistantAgent(
    name = 'PlanningAgent',
    description= 'An agent for planning tasks,this agent should be the first to engage when given a new task.',
    model_client=model_client,
    system_message='''
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are :
        WebSearchAgent : Searches for information.
        DataAnalystAgent : Performs calculations

    You only plan and delegate tasks - you do not exectue them yourself.

    When assigning tasks, use the below format:
    1. <agent> : <task>

    After all the tasks are completed, summarize the findings and end with "TERMINATE"
    ''',
)

In [16]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    system_message='''
        You are a web search agent.
        Your only tool is search_web_tool - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [17]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100


data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.

        If you have not seen the data, ask for it.

    ''',
)

# Termination Conditions

In [18]:
from autogen_agentchat.conditions import MaxMessageTermination,TextMentionTermination

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=10)
combined_termination = text_mention_termination | max_messages_termination

In [19]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

Current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planning agent has assigned task before other agents start working.
Only Select one agent.
'''

In [10]:
'''
Try not to overload the selector prompt with too much information.


selector_prompt (str, optional) – The prompt template to use for selecting the next speaker. 

Available fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. 
1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. 
2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. 

3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.
'''

'\nTry not to overload the selector prompt with too much information.\n\n\nselector_prompt (str, optional) – The prompt template to use for selecting the next speaker. \n\nAvailable fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. \n1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. \n2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. \n\n3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.\n'

In [20]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [21]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [22]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='', arguments='{"query":"Who was the Miami Heat player with the highest points in the 2006-2007 season?"}', name='search_web_tool'), FunctionCall(id='', arguments='{"query":"What was the percentage change in total rebounds for the Miami Heat player with the highest points in the 2006-2007 season between the 2007-2008 and 2008-2009 seasons?"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', name='search_web_tool', call

Error processing publish message for WebSearchAgent_3292495b-d125-4d9d-aa83-13c5fd7ab73d/3292495b-d125-4d9d-aa83-13c5fd7ab73d
Traceback (most recent call last):
  File "/Users/vaibhavarde/Desktop/Autogen/AutogenCrashCourse/.venv/lib/python3.13/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
    )
    ^
  File "/Users/vaibhavarde/Desktop/Autogen/AutogenCrashCourse/.venv/lib/python3.13/site-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vaibhavarde/Desktop/Autogen/AutogenCrashCourse/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/U

RuntimeError: BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'Please ensure that the number of function response parts is equal to the number of function call parts of the function call turn.', 'status': 'INVALID_ARGUMENT'}}]
Traceback:
Traceback (most recent call last):

  File "/Users/vaibhavarde/Desktop/Autogen/AutogenCrashCourse/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_chat_agent_container.py", line 133, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):
    ...<4 lines>...
            await self._log_message(msg)

  File "/Users/vaibhavarde/Desktop/Autogen/AutogenCrashCourse/.venv/lib/python3.13/site-packages/autogen_agentchat/agents/_assistant_agent.py", line 953, in on_messages_stream
    async for inference_output in self._call_llm(
    ...<15 lines>...
            yield inference_output

  File "/Users/vaibhavarde/Desktop/Autogen/AutogenCrashCourse/.venv/lib

# Custom Selector Function

In [23]:
s = "VaibhaV"
s[-1]

'V'

In [24]:
planning_agent.name

'PlanningAgent'

In [25]:
from typing import Sequence
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage

def selector_func(messages : Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if(messages[-1].source != planning_agent.name):
        return planning_agent.name
    return None

In [26]:
await selector_team.reset()
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func)

In [27]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent : Find the Miami Heat player with the highest points scored in the 2006-2007 season.

2. WebSearchAgent : Find the total rebounds for the Miami Heat player with the highest points in the 2006-2007 season for the 2007-2008 season.

3. WebSearchAgent : Find the total rebounds for the same player for the 2008-2009 season.

4. DataAnalystAgent : Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons for the player identified in step 2.

5. WebSearchAgent : Verify the data accuracy for the calculations performed in step 4.

Summarize the findings from the data analysis and the web searches, and then TERMINATE.



TaskResult(messages=[TextMessage(id='c66ba49b-d952-490e-b4e2-0b52b00ed7b7', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 27, 13, 21, 11, 844658, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='0099eb0b-e777-40d6-bd14-a78465ce8649', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=193), metadata={}, created_at=datetime.datetime(2025, 8, 27, 13, 21, 13, 341463, tzinfo=datetime.timezone.utc), content='1. WebSearchAgent : Find the Miami Heat player with the highest points scored in the 2006-2007 season.\n\n2. WebSearchAgent : Find the total rebounds for the Miami Heat player with the highest points in the 2006-2007 season for the 2007-2008 season.\n\n3. WebSearchAgent : Find the total rebounds for the same